In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# !pip install -r requirements.txt

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "xxx"

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
df = pd.read_csv('Input_Data/test_subsampled.csv')
df.head()

,_id,answer,question,supporting_facts,context,type,level
0,5abc0a5d5542993f40c73c64,no,Are Freakonomics and In the Realm of the Hacke...,"[['Freakonomics (film)', 0], ['In the Realm of...","[['Hackers (film)', ['Hackers is a 1995 Americ...",comparison,hard
1,5a760ab65542994ccc918697,Nelson Rockefeller,Alfred Balk served as the secretary of the Com...,"[['Alfred Balk', 0], ['Alfred Balk', 2], ['Nel...","[['Cynthia Hogan', ['Cynthia C. Hogan (born Ci...",bridge,hard
2,5a8a43eb5542996c9b8d5e82,"Marion, South Australia",Which Australian city founded in 1838 contains...,"[['Westminster School, Adelaide', 0], ['Westmi...","[['Sainik School Balachadi, Jamnagar', ['The S...",bridge,hard
3,5ae7ba7a5542993210983f12,Usher,What is the name of the singer who's song was ...,"[[""I Don't Wanna Know"", 1], ['Yeah! (Usher son...","[[""Can't Touch It"", ['""Can\'t Touch It"" is a s...",bridge,hard
4,5aba7cfe554299232ef4a2fd,Carabao Cup,The 2017–18 Wigan Athletic F.C. season will be...,"[['2017–18 Wigan Athletic F.C. season', 1], ['...","[['2017–18 Wigan Athletic F.C. season', [""The ...",bridge,hard


In [8]:
df = df[['question','context','answer']]
df = df.sample(frac=1, random_state=42)
df = df.reset_index(drop=True)
df = pd.DataFrame(df)

In [9]:
import tiktoken
import ast
# Function to count tokens in a string using tiktoken
def count_tokens(text):
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))

# Function to safely parse context - handles both string and list formats
def parse_context(context):
    """
    Parse context whether it's stored as string representation of list or actual list
    """
    if isinstance(context, str):
        try:
            # Try to parse as literal (for string representations of lists)
            parsed = ast.literal_eval(context)
            return parsed
        except (ValueError, SyntaxError):
            # If it's just a plain string, return as is
            return context
    else:
        # If it's already a list or other structure, return as is
        return context

# Function to count tokens for nested context structure
def count_context_tokens(context):
    """
    Count tokens in context, handling different possible formats
    """
    total_tokens = 0
    
    # First parse the context
    parsed_context = parse_context(context)
    
    # Handle different formats
    if isinstance(parsed_context, list):
        for item in parsed_context:
            if isinstance(item, (list, tuple)) and len(item) >= 2:
                # Expected format: [title, text_snippets]
                title = item[0]
                text_snippets = item[1]
                
                # Count tokens in title
                if isinstance(title, str):
                    total_tokens += count_tokens(title)
                
                # Count tokens in text snippets
                if isinstance(text_snippets, list):
                    for snippet in text_snippets:
                        if isinstance(snippet, str):
                            total_tokens += count_tokens(snippet)
                elif isinstance(text_snippets, str):
                    total_tokens += count_tokens(text_snippets)
            else:
                # Handle single items or unexpected formats
                if isinstance(item, str):
                    total_tokens += count_tokens(item)
    elif isinstance(parsed_context, str):
        # If it's just a string, count tokens directly
        total_tokens += count_tokens(parsed_context)
    
    return total_tokens


# Apply the count_context_tokens function to the 'context' column
df['context_token_count'] = df['context'].apply(count_context_tokens)

In [10]:
df

,question,context,answer,context_token_count
0,Cagliostro-Walzer op.370 was composed by a man...,"[['Jiroemon Kimura', ['Jiroemon Kimura (木村 次郎右...",1825,1754
1,What album did John Lennon release before the ...,"[['Instant Karma: All-Time Greatest Hits', [""I...",John Lennon/Plastic Ono Band,1350
2,Which actor in The Lieutenant Wears Skirts is ...,"[['The Lieutenant Wore Skirts', ['The Lieutena...",Tom Ewell,1050
3,"In what year was the Sayrevill, New Jersey roc...","[['The Rats (American band)', ['The Rats were ...",1983,1516
4,"Which genus has the most number of species, My...","[['Styrax crotonoides', ['Styrax crotonoides i...",Myrsine,824
...,...,...,...,...
495,Luke Rockhold defeated the MMA fighter who was...,"[['Kamen Georgiev', ['Kamen Georgiev (Bulgaria...",Anderson Silva,1559
496,"Who was born first, Brooklyn Decker or Andy Ro...","[['Brooklyn Decker', ['Brooklyn Danielle Decke...",Andrew Stephen Roddick,1339
497,Which Outer Banks landmark and premiere BBQ re...,"[[""The Park's Finest"", ['The Park’s Finest is ...",Pigman's Bar-B-Que,1178
498,What is the name of the largest shipping facil...,"[['Transportation in Boston', ['The Boston tra...",Port of Boston,1227


In [11]:
# Find the maximum token count
max_tokens = df['context_token_count'].max()

In [12]:
# Function to flatten context for processing
def flatten_context(context):
    """
    Flatten context into a single string, handling different formats
    """
    parsed_context = parse_context(context)
    
    if isinstance(parsed_context, list):
        flattened_parts = []
        for item in parsed_context:
            if isinstance(item, (list, tuple)) and len(item) >= 2:
                # Expected format: [title, text_snippets]
                title = item[0]
                text_snippets = item[1]
                
                # Add title
                if isinstance(title, str):
                    flattened_parts.append(title)
                
                # Add text snippets
                if isinstance(text_snippets, list):
                    for snippet in text_snippets:
                        if isinstance(snippet, str):
                            flattened_parts.append(snippet)
                elif isinstance(text_snippets, str):
                    flattened_parts.append(text_snippets)
            else:
                # Handle single items
                if isinstance(item, str):
                    flattened_parts.append(item)
        
        return " ".join(flattened_parts)
    elif isinstance(parsed_context, str):
        return parsed_context
    else:
        return str(parsed_context)

df['raw_context'] = df['context'].apply(flatten_context)

In [13]:
# import matplotlib.pyplot as plt
# df.context_token_count.hist(bins=100)
# plt.show()

In [14]:
df

,question,context,answer,context_token_count,raw_context
0,Cagliostro-Walzer op.370 was composed by a man...,"[['Jiroemon Kimura', ['Jiroemon Kimura (木村 次郎右...",1825,1754,"Jiroemon Kimura Jiroemon Kimura (木村 次郎右衛門 , Ki..."
1,What album did John Lennon release before the ...,"[['Instant Karma: All-Time Greatest Hits', [""I...",John Lennon/Plastic Ono Band,1350,Instant Karma: All-Time Greatest Hits Instant ...
2,Which actor in The Lieutenant Wears Skirts is ...,"[['The Lieutenant Wore Skirts', ['The Lieutena...",Tom Ewell,1050,The Lieutenant Wore Skirts The Lieutenant Wore...
3,"In what year was the Sayrevill, New Jersey roc...","[['The Rats (American band)', ['The Rats were ...",1983,1516,The Rats (American band) The Rats were an Amer...
4,"Which genus has the most number of species, My...","[['Styrax crotonoides', ['Styrax crotonoides i...",Myrsine,824,Styrax crotonoides Styrax crotonoides is a spe...
...,...,...,...,...,...
495,Luke Rockhold defeated the MMA fighter who was...,"[['Kamen Georgiev', ['Kamen Georgiev (Bulgaria...",Anderson Silva,1559,Kamen Georgiev Kamen Georgiev (Bulgarian: Каме...
496,"Who was born first, Brooklyn Decker or Andy Ro...","[['Brooklyn Decker', ['Brooklyn Danielle Decke...",Andrew Stephen Roddick,1339,Brooklyn Decker Brooklyn Danielle Decker Roddi...
497,Which Outer Banks landmark and premiere BBQ re...,"[[""The Park's Finest"", ['The Park’s Finest is ...",Pigman's Bar-B-Que,1178,The Park's Finest The Park’s Finest is a Filip...
498,What is the name of the largest shipping facil...,"[['Transportation in Boston', ['The Boston tra...",Port of Boston,1227,Transportation in Boston The Boston transporta...


In [15]:
# Function to create chunks with overlap while preserving sentence boundaries
def create_chunks(text, chunk_size=256, overlap=64):
    # Split text into sentences using newlines
    sentences = text.split('\n')

    chunks = []
    current_chunk = []
    current_length = 0

    i = 0
    while i < len(sentences):
        sentence = sentences[i]
        sentence_length = count_tokens(sentence)

        # If adding this sentence would exceed chunk size, finalize current chunk
        if current_length + sentence_length > chunk_size and current_chunk:
            # Join current chunk sentences
            chunk_text = '\n'.join(current_chunk)
            chunks.append(chunk_text)

            # Create overlap for next chunk
            overlap_chunk = []
            overlap_length = 0

            # Go backwards from current position to create overlap
            j = len(current_chunk) - 1
            while j >= 0 and overlap_length < overlap:
                overlap_sentence = current_chunk[j]
                overlap_sentence_length = count_tokens(overlap_sentence)

                if overlap_length + overlap_sentence_length <= overlap:
                    overlap_chunk.insert(0, overlap_sentence)
                    overlap_length += overlap_sentence_length
                    j -= 1
                else:
                    break

            # Start new chunk with overlap
            current_chunk = overlap_chunk
            current_length = overlap_length

        # Add current sentence to chunk
        current_chunk.append(sentence)
        current_length += sentence_length
        i += 1

    # Add the last chunk if it has content
    if current_chunk:
        chunk_text = '\n'.join(current_chunk)
        chunks.append(chunk_text)

    # Join chunks with '<c>' separator
    return '<c>'.join(chunks)

# Create the chunks column
df['chunks'] = df['raw_context'].apply(create_chunks)

In [16]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm
import time
import os

# Configuration
PERSIST_DIRECTORY = "Input_Data/HotpotQA_dataset_test/chroma_db_all-MiniLM-L6-v2"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# Instantiate Hugging Face embeddings on cuda:0
hf_embed_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL, model_kwargs={"device": "cuda:0"})

def check_vectorstore_exists(persist_dir):
    """
    Check if ChromaDB vectorstore already exists at the given directory
    """
    if not os.path.exists(persist_dir):
        return False

    # Check if the directory contains ChromaDB files
    chroma_files = ['chroma.sqlite3', 'index']
    return any(os.path.exists(os.path.join(persist_dir, file)) for file in chroma_files)

def load_existing_vectorstore(persist_dir, embedding_function):
    """
    Load existing ChromaDB vectorstore
    """
    print(f"Loading existing ChromaDB from {persist_dir}...")
    vectorstore = Chroma(
        embedding_function=embedding_function,
        persist_directory=persist_dir
    )

    collection_count = vectorstore._collection.count()
    print(f"Loaded existing ChromaDB with {collection_count} documents")
    return vectorstore

def create_new_vectorstore(df, persist_dir, embedding_function):
    """
    Create new ChromaDB vectorstore from DataFrame
    """
    # Prepare data for ChromaDB
    documents = []
    metadatas = []
    ids = []

    print("Processing documents and creating chunks...")
    print(f"Total rows to process: {len(df)}")

    # Extract chunks and create documents with progress tracking
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        chunks = row['chunks'].split('<c>')

        for chunk_idx, chunk in enumerate(chunks):
            # Skip empty chunks
            if chunk.strip():
                documents.append(chunk.strip())

                # Create metadata for each chunk
                metadata = {
                    'source_index': idx,
                    'chunk_index': chunk_idx,
                    'total_chunks': len(chunks),
                    'token_count': count_tokens(chunk.strip())
                }
                metadatas.append(metadata)

                # Create unique ID for each chunk
                chunk_id = f"doc_{idx}_chunk_{chunk_idx}"
                ids.append(chunk_id)

    print(f"\nTotal chunks prepared: {len(documents)}")
    print(f"Average chunks per document: {len(documents)/len(df):.2f}")

    # Create ChromaDB instance
    print("Creating new ChromaDB instance...")
    vectorstore = Chroma(
        embedding_function=embedding_function,
        persist_directory=persist_dir
    )

    # Add documents to ChromaDB with progress tracking
    print("Adding documents to ChromaDB...")
    batch_size = 100  # Process in batches to show progress and manage memory

    start_time = time.time()

    for i in tqdm(range(0, len(documents), batch_size), desc="Adding batches to ChromaDB"):
        end_idx = min(i + batch_size, len(documents))

        batch_documents = documents[i:end_idx]
        batch_metadatas = metadatas[i:end_idx]
        batch_ids = ids[i:end_idx]

        vectorstore.add_texts(
            texts=batch_documents,
            metadatas=batch_metadatas,
            ids=batch_ids
        )

        # Optional: Print progress every few batches
        if (i // batch_size + 1) % 10 == 0:
            elapsed = time.time() - start_time
            processed = end_idx
            rate = processed / elapsed
            eta = (len(documents) - processed) / rate if rate > 0 else 0
            print(f"  Processed {processed}/{len(documents)} documents ({processed/len(documents)*100:.1f}%) | Rate: {rate:.1f} docs/sec | ETA: {eta/60:.1f} min")

    total_time = time.time() - start_time
    print(f"\nCompleted! Total time: {total_time/60:.2f} minutes")
    print(f"Average processing rate: {len(documents)/total_time:.1f} documents per second")
    print(f"ChromaDB created with {len(documents)} document chunks")

    return vectorstore

# Main execution logic
print("Checking if ChromaDB vectorstore already exists...")

if check_vectorstore_exists(PERSIST_DIRECTORY):
    # Load existing vectorstore
    vectorstore = load_existing_vectorstore(PERSIST_DIRECTORY, hf_embed_model)
    print("✅ Using existing ChromaDB vectorstore")

else:
    # Create new vectorstore
    print("❌ ChromaDB vectorstore not found. Creating new one...")
    vectorstore = create_new_vectorstore(df, PERSIST_DIRECTORY, hf_embed_model)
    print("✅ New ChromaDB vectorstore created successfully")

# Optional: Verify the database
collection_count = vectorstore._collection.count()
print(f"Final verification: ChromaDB collection contains {collection_count} documents")

# Optional: Add a force recreation option
def recreate_vectorstore_if_needed(force_recreate=False):
    """
    Utility function to force recreation of vectorstore if needed
    """
    if force_recreate:
        print("🔄 Force recreation requested...")
        # Remove existing directory if it exists
        if os.path.exists(PERSIST_DIRECTORY):
            import shutil
            shutil.rmtree(PERSIST_DIRECTORY)
            print(f"Removed existing vectorstore at {PERSIST_DIRECTORY}")

        # Create new vectorstore
        return create_new_vectorstore(df, PERSIST_DIRECTORY, hf_embed_model)

    return vectorstore

# Uncomment the line below if you want to force recreation
# vectorstore = recreate_vectorstore_if_needed(force_recreate=True)

/tmp/ipykernel_1411879/3507626788.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embed_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL, model_kwargs={"device": "cuda:0"})
2025-10-27 13:42:52.332373: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Checking if ChromaDB vectorstore already exists...
❌ ChromaDB vectorstore not found. Creating new one...
Processing documents and creating chunks...
Total rows to process: 500


Processing rows: 100%|██████████| 500/500 [00:00<00:00, 1981.46it/s]



Total chunks prepared: 500
Average chunks per document: 1.00
Creating new ChromaDB instance...
Adding documents to ChromaDB...


Adding batches to ChromaDB: 100%|██████████| 5/5 [00:01<00:00,  2.74it/s]


Completed! Total time: 0.03 minutes
Average processing rate: 273.3 documents per second
ChromaDB created with 500 document chunks
✅ New ChromaDB vectorstore created successfully
Final verification: ChromaDB collection contains 500 documents


In [17]:
def retrieve_documents(query, vectorstore=vectorstore, k=3):
    relevant_docs = vectorstore.similarity_search(query, k=k)
    return relevant_docs


In [19]:
# !pip install accelerate

In [20]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
hllm = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
from transformers import pipeline

# Create a pipeline for text-generation
hllm = pipeline("text-generation", model=hllm, tokenizer=tokenizer,pad_token_id=tokenizer.eos_token_id)


Device set to use cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 44.34 GiB of which 181.31 MiB is free. Process 295024 has 1.34 GiB memory in use. Process 696622 has 39.07 GiB memory in use. Including non-PyTorch memory, this process has 3.72 GiB memory in use. Of the allocated memory 3.39 GiB is allocated by PyTorch, and 25.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def route_question(query, hllm):
    
    # Compose the prompt as a single string (no chat format)
    prompt = f"""You are a question classifier. Your job is to analyze the question and classify it into one of the following three categories:

LLM – If the question can be answered using general knowledge the LLM has been trained on, without needing retrieval.

Single-hop – If the answer requires retrieving a single fact or document to answer.

Multi-hop – If the answer requires combining multiple pieces of information retrieved from different documents or sources.

Return only one of the three labels: LLM, Single-hop, or Multi-hop.

Examples:
Question: What is the capital of France?
Answer: LLM

Question: Who is the CEO of OpenAI?
Answer: LLM

Question: What is the average temperature in Berlin in June?
Answer: Single-hop

Question: When was the iPhone 12 released and how did its sales compare to the iPhone 11?
Answer: Multi-hop

Question: What is the boiling point of water at sea level?
Answer: LLM

Question: What are the side effects of ibuprofen?
Answer: Single-hop

Question: Compare the GDP of Germany and France in 2023.
Answer: Multi-hop

Question: What year did World War II end?
Answer: LLM

Question: Who won the best actor Oscar in 2024 and what movie were they in?
Answer: Multi-hop

Question: How many calories are in a McDonald's Big Mac?
Answer: Single-hop

Now classify the following question:
Question: {query}
Answer:"""

    # Generate the classification
    output = hllm(prompt, max_new_tokens=100, do_sample=False)[0]['generated_text']

    # Extract only the answer label (last line after "Answer:")
    answer = output.split("Answer:")[-1].strip().split("\n")[0]
    answer = answer.replace('"', '').replace("'", "")
    return answer

In [22]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B")
# # llm = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B")

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B",device_map = 'cuda:0')
# llm = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B",device_map = 'cuda:0')


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl",device_map='cuda:1')
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl").to('cuda:1')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 252.00 MiB. GPU 0 has a total capacity of 44.34 GiB of which 181.31 MiB is free. Process 295024 has 1.34 GiB memory in use. Process 696622 has 39.07 GiB memory in use. Including non-PyTorch memory, this process has 3.72 GiB memory in use. Of the allocated memory 3.39 GiB is allocated by PyTorch, and 25.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [19]:
def answer_generator_direct(query):
    # Use the globally loaded tokenizer and llm
    # Prepare the prompt for the model
    prompt = f"User question: {query}\n Answer:"

    # Tokenize and generate, passing attention_mask for reliable results
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda:0")
    attention_mask = inputs.attention_mask.to("cuda:0")

    pad_token_id = tokenizer.eos_token_id

    output_ids = llm.generate(input_ids, attention_mask=attention_mask, pad_token_id=pad_token_id)
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract only the answer after "Answer:"
    if "Answer:" in output:
        answer = output.split("Answer:")[-1].strip().split("\n")[0]
    else:
        answer = output.strip().split("\n")[-1]
    answer = answer.replace('"', '').replace("'", "")
    return answer

In [20]:
def answer_generator_single_hop(query, context):
    # Use the globally loaded tokenizer and llm from file_context_1
    # Prepare the prompt for the model
    prompt = (
        "You are an expert at answering the question just based on the context. "
        "Given the context, answer the user question. If you cannot answer the question based on context, "
        "state properly that you cannot answer the question.\n\n"
        f"Context:\n{context}\n\nUser question: {query}\nAnswer:"
    )

    # Tokenize and generate, passing attention_mask for reliable results
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda:0")
    attention_mask = inputs.attention_mask.to("cuda:0")

    pad_token_id = tokenizer.eos_token_id

    output_ids = llm.generate(input_ids, attention_mask=attention_mask, pad_token_id=pad_token_id)
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract only the answer after "Answer:"
    if "Answer:" in output:
        answer = output.split("Answer:")[-1].strip().split("\n")[0]
    else:
        answer = output.strip().split("\n")[-1]
    answer = answer.replace('"', '').replace("'", "")
    return answer

In [21]:
def chain_of_thought_retriever(query, vectorstore, answer_generator_func, max_iterations=3, k=3):

    def create_cot_prompt(query, context):
        return (
            "You are an expert at step-by-step reasoning. "
            "Answer the following question by reasoning step-by-step based on the provided context.Strictly follow the instructions please.\n\n"
            f"Context:\n{context}\n\n"
            f"Question: {query}\n\n"
            "Instructions:\n"
            "1. Break down the question into logical steps\n"
            "2. Use the context to find relevant information for each step\n"
            "3. Reason through each step systematically\n"
            "4. If you can provide a complete answer, conclude with \"So the answer is: [your answer]\"\n"
            "5. If you need more information to answer completely, state what additional information is needed and provide your current reasoning\n\n"
            "Answer by reasoning step-by-step:"
        )

    def extract_reasoning_and_check_completeness(response):
        response = response.strip()
        
        # Check for the complete answer pattern
        if "So the answer is:" in response:
            answer_part = response.split("So the answer is:")[-1].strip()
            return {
                "complete": True,
                "answer": answer_part,
                "reasoning": response
            }
        
        # If response is very short and seems like a direct answer, treat as complete
        response_lower = response.lower()
        if (len(response.split()) <= 5 and 
            not any(phrase in response_lower for phrase in [
                "need more information", "cannot determine", "missing", 
                "unclear", "require", "cannot find", "not enough", 
                "insufficient", "unable to", "more details"
            ])):
            # This looks like a direct answer, format it properly
            return {
                "complete": True,
                "answer": response,
                "reasoning": f"Based on the context provided, the answer is: {response}. So the answer is: {response}"
            }
        
        # Otherwise, it's incomplete
        return {
            "complete": False,
            "answer": None,
            "reasoning": response
        }

    def generate_follow_up_query(reasoning, original_query):
        reasoning_lower = reasoning.lower()
        # Look for phrases indicating missing information
        if any(phrase in reasoning_lower for phrase in ["need more information", "cannot determine", "missing", "unclear", "require", "cannot find"]):
            lines = reasoning.split('\n')
            for line in lines:
                if any(phrase in line.lower() for phrase in ["need", "require", "missing", "unclear", "cannot find"]):
                    words = line.split()
                    key_terms = [word.strip('.,?!') for word in words if len(word) > 3 and word.isalpha()]
                    if key_terms:
                        return " ".join(key_terms[:5])
        # Fallback: extract key entities from original query
        query_words = original_query.split()
        return " ".join([word for word in query_words if len(word) > 3])[:100]

    all_contexts = []
    reasoning_steps = []
    iteration_details = []
    current_query = query

    for iteration in range(1, max_iterations + 1):
        try:
            relevant_docs = vectorstore.similarity_search(current_query, k=k)
            current_context = "\n\n".join([
                f"Document {i+1}:\n{doc.page_content}"
                for i, doc in enumerate(relevant_docs)
            ])
            all_contexts.append({
                "iteration": iteration,
                "query": current_query,
                "context": current_context,
                "num_docs": len(relevant_docs)
            })
        except Exception as e:
            return {
                "success": False,
                "error": f"Document retrieval failed at iteration {iteration}: {e}",
                "final_answer": None,
                "reasoning_steps": reasoning_steps,
                "iteration_details": iteration_details,
                "all_contexts": all_contexts
            }

        # Step 2: Create CoT prompt and generate reasoning
        cot_prompt = create_cot_prompt(query, current_context)
        try:
            reasoning_response = answer_generator_func(cot_prompt, current_context)  # Pass the cot_prompt, not original query
            reasoning_steps.append({
                "iteration": iteration,
                "query": current_query,
                "reasoning": reasoning_response
            })
        except Exception as e:
            return {
                "success": False,
                "error": f"Reasoning generation failed at iteration {iteration}: {e}",
                "final_answer": None,
                "reasoning_steps": reasoning_steps,
                "iteration_details": iteration_details,
                "all_contexts": all_contexts
            }

        # Step 3: Check if reasoning is complete
        result = extract_reasoning_and_check_completeness(reasoning_response)
        iteration_details.append({
            "iteration": iteration,
            "query": current_query,
            "num_docs_retrieved": len(relevant_docs),
            "reasoning_complete": result["complete"],
            "reasoning_length": len(reasoning_response)
        })

        if result["complete"]:
            return {
                "success": True,
                "final_answer": result["answer"],
                "complete_reasoning": result["reasoning"],
                "iterations_used": iteration,
                "reasoning_steps": reasoning_steps,
                "iteration_details": iteration_details,
                "all_contexts": all_contexts
            }

        # Step 4: Generate follow-up query for next iteration
        if iteration < max_iterations:
            current_query = generate_follow_up_query(reasoning_response, query)

    # Return incomplete result
    return {
        "success": False,
        "final_answer": None,
        "error": f"Could not generate complete answer within {max_iterations} iterations",
        "last_reasoning": reasoning_steps[-1]["reasoning"] if reasoning_steps else None,
        "iterations_used": max_iterations,
        "reasoning_steps": reasoning_steps,
        "iteration_details": iteration_details,
        "all_contexts": all_contexts
    }


In [22]:
def answer_generator_multi_hop(query, vectorstore, answer_generator, k=3):

    cot_result = chain_of_thought_retriever(
        query=query,
        vectorstore=vectorstore,
        answer_generator_func=answer_generator,
        max_iterations=3,
        k=k
    )

    all_documents = []
    for context_info in cot_result.get('all_contexts', []):
        all_documents.append({
            "iteration": context_info['iteration'],
            "content": context_info['context'],
            "query_used": context_info['query']
        })

    return {
        "documents": all_documents,
        "cot_result": cot_result,
        "final_answer": cot_result.get('final_answer'),
        "success": cot_result.get('success', False)
    }

In [26]:
result = answer_generator_multi_hop(
    query="""The Oberoi family is part of a hotel company that has a head office in what city?""",
    vectorstore=vectorstore,
    answer_generator=answer_generator_single_hop,
    k=1
)

if result['success']:
    print(f"Answer: {result['final_answer']}")
else:
    print("Could not find complete answer")


Answer: None


In [23]:
from typing import Dict, Any, List, Union
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from langchain.schema import Document

# Define the state that will be passed between nodes
class RoutingRAGState(TypedDict):
    question: str
    original_question: str
    routing_result: str
    answer: str
    answer_type: str  # "direct", "single-hop", or "multi-hop"
    # Metadata for tracking the path taken
    path_taken: List[str]
    error_message: str
    context_docs: List[Document]

def create_routing_rag_graph(
    vectorstore,
    router_llm,
    answer_generation_direct,
    answer_generation_single_hop,
    answer_generation_multi_hop,
    route_question,
    k: int = 3
):
    
    def route_question_node(state: RoutingRAGState) -> RoutingRAGState:

        try:
            routing_decision = route_question(state["question"], router_llm)

            return {
                **state,
                "routing_result": routing_decision,
                "path_taken": state["path_taken"] + [f"route_question -> {routing_decision}"]
            }

        except Exception as e:
            return {
                **state,
                "routing_result": "single-hop",  # Default fallback
                "error_message": f"Routing error: {e}",
                "path_taken": state["path_taken"] + ["route_question -> error_fallback"]
            }

    def answer_generation_direct_node(state: RoutingRAGState) -> RoutingRAGState:

        try:
            answer = answer_generation_direct(state["question"])
            print(f"✅ Direct answer generated: {answer}...")

            return {
                **state,
                "answer": answer,
                "answer_type": "direct",
                "path_taken": state["path_taken"] + ["answer_generation_direct"]
            }

        except Exception as e:
            return {
                **state,
                "answer": f"I apologize, but I encountered an error generating a direct answer: {e}",
                "answer_type": "direct_error",
                "error_message": f"Direct answer error: {e}",
                "path_taken": state["path_taken"] + ["answer_generation_direct -> error"]
            }

    def answer_generation_single_hop_node(state: RoutingRAGState) -> RoutingRAGState:

        try:
            context = vectorstore.similarity_search(state["question"], k=3)
            answer = answer_generation_single_hop(state["question"], context)

            return {
                **state,
                "answer": answer,
                "answer_type": "single-hop",
                "context_docs": context,
                "path_taken": state["path_taken"] + ["answer_generation_single_hop"]
            }

        except Exception as e:
            return {
                **state,
                "answer": f"I apologize, but I encountered an error during single-hop retrieval: {e}",
                "answer_type": "single-hop_error",
                "error_message": f"Single-hop error: {e}",
                "path_taken": state["path_taken"] + ["answer_generation_single_hop -> error"]
            }

    def answer_generation_multi_hop_node(state: RoutingRAGState) -> RoutingRAGState:
        """Generate answer using your existing multi-hop function"""

        try:
            result = answer_generation_multi_hop(state["question"], vectorstore, answer_generator_single_hop, k)

            return {
                **state,
                "answer": result['final_answer'],
                "answer_type": "multi-hop",
                "context_docs": result['documents'], 
                "path_taken": state["path_taken"] + ["answer_generation_multi_hop"]
            }

        except Exception as e:
            return {
                **state,
                "answer": f"I apologize, but I encountered an error during multi-hop retrieval: {e}",
                "answer_type": "multi-hop_error",
                "error_message": f"Multi-hop error: {e}",
                "path_taken": state["path_taken"] + ["answer_generation_multi_hop -> error"]
            }

    def final_answer_node(state: RoutingRAGState) -> RoutingRAGState:

        if state.get('error_message'):
            print(f"⚠️ Warnings: {state['error_message']}")

        return {
            **state,
            "path_taken": state["path_taken"] + ["final_answer"]
        }

    # Conditional routing function
    def decide_answer_generation_strategy(state: RoutingRAGState) -> str:
        """Route to appropriate answer generation strategy based on routing result"""
        routing_result = state["routing_result"].lower()

        if "llm" in routing_result:
            return "answer_generation_direct"
        elif "single-hop" in routing_result or "single" in routing_result:
            return "answer_generation_single_hop"
        else:  # multi-hop or any other case
            return "answer_generation_multi_hop"

    # Build the graph
    workflow = StateGraph(RoutingRAGState)

    # Add nodes
    workflow.add_node("route_question", route_question_node)
    workflow.add_node("answer_generation_direct", answer_generation_direct_node)
    workflow.add_node("answer_generation_single_hop", answer_generation_single_hop_node)
    workflow.add_node("answer_generation_multi_hop", answer_generation_multi_hop_node)
    workflow.add_node("final_answer", final_answer_node)

    # Set entry point
    workflow.set_entry_point("route_question")

    # Add conditional edge from routing to answer generation
    workflow.add_conditional_edges(
        "route_question",
        decide_answer_generation_strategy,
        {
            "answer_generation_direct": "answer_generation_direct",
            "answer_generation_single_hop": "answer_generation_single_hop",
            "answer_generation_multi_hop": "answer_generation_multi_hop"
        }
    )

    # All answer generation nodes lead to final answer
    workflow.add_edge("answer_generation_direct", "final_answer")
    workflow.add_edge("answer_generation_single_hop", "final_answer")
    workflow.add_edge("answer_generation_multi_hop", "final_answer")

    # Terminal node
    workflow.add_edge("final_answer", END)

    return workflow.compile()


def run_routing_rag_workflow(
    question: str,
    compiled_graph
) -> Dict[str, Any]:

    initial_state = RoutingRAGState(
        question=question,
        original_question=question,
        routing_result="",
        answer="",
        answer_type="",
        path_taken=[],
        error_message="",
        context_docs=[]
    )

    final_state = compiled_graph.invoke(initial_state)

    return final_state


def setup_and_run_routing_rag(
    question: str,
    vectorstore,
    router_llm,
    answer_generation_direct,
    answer_generation_single_hop,
    answer_generation_multi_hop,
    route_question,
    k: int = 3
):
    
    compiled_graph = create_routing_rag_graph(
        vectorstore=vectorstore,
        router_llm=router_llm,
        answer_generation_direct=answer_generation_direct,
        answer_generation_single_hop=answer_generation_single_hop,
        answer_generation_multi_hop=answer_generation_multi_hop,
        route_question=route_question,
        k=k
    )

    result = run_routing_rag_workflow(question, compiled_graph)

    return result



In [24]:
def main(question):

    result = setup_and_run_routing_rag(
        question=question,
        vectorstore=vectorstore,
        router_llm=hllm,
        answer_generation_direct=answer_generator_direct,
        answer_generation_single_hop=answer_generator_single_hop,
        answer_generation_multi_hop=answer_generator_multi_hop,
        route_question=route_question,
        k=3
    )

    return result

In [25]:
import time
from tqdm import tqdm

answers = []
paths = []
contexts = []
times = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing questions"):
    question = row['question']
    stime = time.time()
    result = main(question)
    times.append(time.time() - stime)
    answers.append(result.get('answer', None))
    paths.append(result.get('path_taken', None))
    contexts.append(result.get('context_docs', None))

df['predicted_answer'] = answers
df['path_taken'] = paths
df['retrieved_context'] = contexts
df['time'] = times

Processing questions:  60%|██████    | 6/10 [00:56<00:31,  7.85s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Direct answer generated: Portland State University and Boston University are both research institutions. Portland State University is a public research university in...


Processing questions:  70%|███████   | 7/10 [00:59<00:19,  6.36s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Processing questions: 100%|██████████| 10/10 [01:35<00:00,  9.60s/it]


In [27]:
df

,question,context,answer,predicted_answer,path_taken,retrieved_context,time
0,"Jared Leto is an American entertainer, who has...","[['Hurricane (Thirty Seconds to Mars song)', [...","Steve Roland ""Pre"" Prefontaine",Steve Prefontaine,"[route_question -> Multi-hop, answer_generatio...","[{'iteration': 1, 'content': 'Document 1: Hurr...",17.189109
1,ESPN College Football Friday Primetime is prec...,"[['Derek Carr', ['Derek Dallas Carr (born Marc...",Daniel Kanell,Tim Hasselbeck,"[route_question -> Single-hop, answer_generati...",[page_content='Derek Carr Derek Dallas Carr (b...,3.111104
2,This Serbian-American inventor was a person yo...,"[['Recluse', ['A recluse is a person who lives...",the public and society,Nikola Tesla,"[route_question -> Multi-hop, answer_generatio...","[{'iteration': 1, 'content': 'Document 1: Recl...",15.325993
3,When was the movie adaption of Jennifer Weiner...,"[['Jennifer Weiner', ['Jennifer Weiner (born M...",2005,In Her Shoes is a 2005 American comedy-drama f...,"[route_question -> Single-hop, answer_generati...",[page_content='Jennifer Weiner Jennifer Weiner...,3.269786
4,Operation Hedge-hop was a military plan develo...,"[['McDonnell FH Phantom', ['The McDonnell FH P...",United States Army Air Forces (USAAF),None,"[route_question -> Multi-hop, answer_generatio...","[{'iteration': 1, 'content': 'Document 1: McDo...",15.491253
5,Portland State University and Boston Universit...,"[['Portland State University', ['Portland Stat...",yes,Portland State University and Boston Universit...,"[route_question -> LLM (Yes, both Portland Sta...",[],1.758760
6,Agent Provocateur contained the biggest single...,"[['Robert Poley', ['Robert Poley, or Pooley (f...",November 1984,November,"[route_question -> Single-hop, answer_generati...","[page_content='Robert Poley Robert Poley, or P...",3.287352
7,"What class was 6029,a four-cylinder, simple, n...","[['Hunter Valley Mining Locomotives', ['Hunter...",The AD60 class,None,"[route_question -> Multi-hop, answer_generatio...","[{'iteration': 1, 'content': 'Document 1: Hunt...",15.941521
8,The 2003 Major League Soccer All-Star Game was...,"[['2010 MLS All-Star Game', ['The 2010 Major L...",StubHub Center,StubHub Center,"[route_question -> Multi-hop, answer_generatio...","[{'iteration': 1, 'content': 'Document 1: 2010...",10.040618
9,The GUID Partition Table forms a part of which...,"[['Volume Table of Contents', ['In the IBM mai...",Unified Extensible Firmware Interface,Unified Extensible Firmware Interface (UEFI),"[route_question -> Multi-hop, answer_generatio...","[{'iteration': 1, 'content': 'Document 1: Volu...",10.536506


In [28]:
df.to_csv("Adaptive_RAG_on_HotpotQA_dataset_Llama3B.csv")